In [2]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import certifi
import ssl
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
context = ssl.create_default_context(cafile=certifi.where())



C:\Users\Tomasz Skrzyp\AppData\Local\Temp\ipykernel_13156\1788397886.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def set_values(s,e):
    start=s
    end=e
    return start,end
start,end=set_values(1997,2024)
print(start)
    

1997


In [65]:
# define urls for per game and advanced stats
#extracting data from site
def get_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    
 
    data = []
    if url[-13:]=="advanced.html":
        headers = [th.text.strip() for th in table.find_all('th') if th.text.strip()!=""][7:]
        for tr in table.find_all('tr')[1:]:
            if str(tr)[:16]=="<tr class=\"thead" or str(tr)[:16]=="<tr class=\"itali":
                continue
            row = np.array([td.text.strip() for td in tr.find_all('td')[6:] if "DUMMY" not in str(td)])
            data.append(row)
    else:
        headers = [th.text.strip() for th in table.find_all('th')][:30]
        
        for tr in table.find_all('tr')[1:-1]:
            
            if str(tr)[:16]=="<tr class=\"thead"or str(tr)[:16]=="<tr class=\"parti" :
                
                data[-1][3]=[td.text.strip() for td in tr.find_all('td') ][2]#in case player played for more teams in a seaosn
                continue
            
            row = np.append([0],np.array([td.text.strip() for td in tr.find_all('td') ]))[:30]
            
            data.append(row)
          
    headers=headers[:row.size]

    arr=np.array(data)
      

    return arr,headers

#creating data frame

per_game_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'
advanced_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'
def data_frame():
    for i in range(start,end+1):
        per_game_for_seas,per_game_headers=get_data(per_game_url.format(i))
        
        per_game_for_seas=np.insert(per_game_for_seas,3,i,axis=1)
        print(per_game_for_seas)
        advanced_for_seas,advanced_headers=get_data(advanced_url.format(i))
        print(advanced_for_seas.shape)
        temp_full=np.concatenate((per_game_for_seas,advanced_for_seas),axis=1)
        if i==1997:
            full=temp_full
        else:
            full=np.concatenate((full,temp_full))
        print(i)
        
    whole_data=pd.DataFrame(full)
    headers=np.concatenate((per_game_headers,advanced_headers))
    headers=np.insert(headers, 3,"Season")
    whole_data.columns=headers
    whole_data.Rk=[i for i in range(1,len(full)+1)]
    return whole_data

   
   

In [66]:
whole_data=data_frame()


nba_abbreviations_teams = {
    'PHO': 'Phoenix Suns',
    'WSB':'Washington Wizards',
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'BRK':'Brooklyn Nets',
    'CHO':'Charlotte Hornets',
    'CHH': 'Charlotte Hornets',
    'NOK': 'New Orleans Pelicans',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards',
    'CHA': 'Charlotte Hornets', # Became Charlotte Hornets in 2014
    'NJN': 'New Jersey Nets',   # Moved to Brooklyn and became Brooklyn Nets in 2012
    'NOH': 'New Orleans Hornets', # Became New Orleans Pelicans in 2013
    'SEA': 'Seattle SuperSonics', # Relocated to Oklahoma City and became Oklahoma City Thunder in 2008
    'VAN': 'Memphis Grizzlies', # Relocated to Memphis and became Memphis Grizzlies in 2001
}
whole_data.to_csv("whole_data.csv")




[['0' 'Michael Jordan' '33' ... '2.0' '1.9' '29.6']
 ['0' 'Karl Malone' '33' ... '2.8' '2.6' '27.4']
 ['0' 'Glen Rice' '29' ... '2.2' '2.4' '26.8']
 ...
 ['0' 'Cuonzo Martin' '25' ... '0.3' '0.3' '0.0']
 ['0' 'Anthony Miller' '25' ... '0.0' '2.0' '0.0']
 ['0' 'Elmore Spencer' '27' ... '1.0' '0.0' '0.0']]
(441, 20)
1997
[['0' 'Michael Jordan' '34' ... '2.3' '1.8' '28.7']
 ['0' "Shaquille O'Neal" '25' ... '2.9' '3.2' '28.3']
 ['0' 'Karl Malone' '34' ... '3.0' '2.9' '27.0']
 ...
 ['0' 'Sean Higgins' '29' ... '0.5' '1.5' '0.0']
 ['0' 'Bob McCann' '33' ... '0.0' '1.0' '0.0']
 ['0' 'Michael McDonald' '28' ... '2.0' '1.0' '0.0']]
(439, 20)
1998
[['0' 'Allen Iverson' '23' ... '3.5' '2.0' '26.8']
 ['0' "Shaquille O'Neal" '26' ... '2.5' '3.2' '26.3']
 ['0' 'Karl Malone' '35' ... '3.3' '2.7' '23.8']
 ...
 ['0' 'Brian Shaw' '32' ... '1.0' '1.0' '0.0']
 ['0' 'Trevor Winter' '25' ... '0.0' '5.0' '0.0']
 ['0' 'Joe Wolf' '34' ... '0.0' '2.0' '0.0']]
(440, 20)
1999
[['0' "Shaquille O'Neal" '27' ... '2.

In [67]:

#splitting the data to numerical and personal
numerical_data=pd.DataFrame(whole_data.values[:,6:],columns=whole_data.columns[6:])

p_id=pd.DataFrame(np.zeros((whole_data.shape[0],8)),columns=["Rk","Player","team", "season","Age","seed","cons_mvp","mvp_votes"])
p_id.Rk=whole_data.Rk
p_id.Age=whole_data.Age
p_id.Player=whole_data.Player
p_id.season=whole_data.Season
p_id.team=whole_data.Team


In [68]:
print(p_id)

          Rk             Player team season Age  seed  cons_mvp  mvp_votes
0          1     Michael Jordan  CHI   1997  33   0.0       0.0        0.0
1          2        Karl Malone  UTA   1997  33   0.0       0.0        0.0
2          3          Glen Rice  CHH   1997  29   0.0       0.0        0.0
3          4   Shaquille O'Neal  LAL   1997  24   0.0       0.0        0.0
4          5     Mitch Richmond  SAC   1997  31   0.0       0.0        0.0
...      ...                ...  ...    ...  ..   ...       ...        ...
13413  13414        Danny Green  PHI   2024  36   0.0       0.0        0.0
13414  13415     Ron Harper Jr.  TOR   2024  23   0.0       0.0        0.0
13415  13416     Justin Jackson  MIN   2024  28   0.0       0.0        0.0
13416  13417  Dmytro Skapintsev  NYK   2024  25   0.0       0.0        0.0
13417  13418      Javonte Smart  PHI   2024  24   0.0       0.0        0.0

[13418 rows x 8 columns]


In [69]:
#filling the standings
def givehtml(s):
    wikiurl="https://en.wikipedia.org/wiki/{}%E2%80%93{}_NBA_season"
    if s<2009 and s>=1999:
        page=urlopen(wikiurl.format(str(s),"0"+str((s+1)%100)))
    
    else:
        page=urlopen(wikiurl.format(str(s),str((s+1)%100)))
    htmllang=page.read()
    htmlstr=htmllang.decode("utf-8")
    

    return htmlstr


    
def cut_data(htmlstr):
    try:
        startEastv1=htmlstr.index("<h3 id=\"By_conference\">By conference</h3>")
    except:
        
        startEastv1=htmlstr.index("<h4 id=\"By_conference\">By conference</h4>")

    startEast=htmlstr[startEastv1:].index("season")+1+startEastv1
    finishEast=htmlstr[startEast:].index("West")+startEast
    startWest=finishEast
    finishWest=htmlstr[startWest:].index("Playoffs")+startWest
    east_data=htmlstr[startEast:finishEast]
    west_data=htmlstr[startWest:finishWest]

    return east_data,west_data


def get_stands(results,start,end):
    for i in range(start-1,end):
        print(i)
        east_data,west_data=cut_data(givehtml(i))
        res_dictionary={}
        #east
       
        start=east_data.find("season")
        n=30
        while start >= 0 and n > 1:
            start = east_data.find("season", start+1)
            if n%2==0:
                  sign=east_data[start:].find("<")+start
                  res_dictionary[east_data[start+8:sign]]=int((32-n)/2)
            n -= 1
        
        #west
        if i>=2004:
            start=west_data.find("season")
            n=30
            while start >= 0 and n > 1:
                start = west_data.find("season", start+1)
                if n%2==0:
                  sign=west_data[start:].find("<")+start
                  res_dictionary[west_data[start+8:sign]]=int((32-n)/2)
                n-=1
        else:
            n=28
            start=west_data.find("season")
      
            while start >= 0 and n > 1:
                start = west_data.find("season", start+1)
                if n%2==0:
                  sign=west_data[start:].find("<")+start
                  res_dictionary[west_data[start+8:sign]]=int((30-n)/2)
                n-=1
        results.append(res_dictionary)        

     
    
        
#changing past team names
results=[] 
get_stands(results,start,end)


for i in results:
    try:
       val=i["Washington Bullets"]
       del i["Washington Bullets"]
       i["Washington Wizards"]=val
    except:
         pass
    
    try:
        val=i['Vancouver Grizzlies']
        del i['Vancouver Grizzlies']
        i['Memphis Grizzlies']=val
    except:pass
    try:
        val=i['Charlotte Bobcats']
        del i['Charlotte Bobcats']
        i['Charlotte Hornets']=val
    except:pass
    try:
        val=i['New Orleans/Oklahoma City Hornets']
        del i['New Orleans/Oklahoma City Hornets']
        i['New Orleans Pelicans']=val
    except:pass
    
  
#seeding
for i,v in enumerate(p_id.values):
       
    
            p_id.at[i,"seed"]=float(results[int(v[3])-start][nba_abbreviations_teams[v[2]]])
#in case i didn't want to do the mvp voting data part(only for current year)
if p_id.shape[0]<1000:
    combined=pd.DataFrame(np.concatenate((p_id.values,numerical_data),axis=1),columns=np.append(p_id.columns,numerical_data.columns))
    combined.to_csv("data/23-24data.csv")    
            
    
        

        




1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
          Rk             Player team season Age  seed  cons_mvp  mvp_votes
0          1     Michael Jordan  CHI   1997  33   1.0       0.0        0.0
1          2        Karl Malone  UTA   1997  33   1.0       0.0        0.0
2          3          Glen Rice  CHH   1997  29   6.0       0.0        0.0
3          4   Shaquille O'Neal  LAL   1997  24   4.0       0.0        0.0
4          5     Mitch Richmond  SAC   1997  31   9.0       0.0        0.0
...      ...                ...  ...    ...  ..   ...       ...        ...
13413  13414        Danny Green  PHI   2024  36   7.0       0.0        0.0
13414  13415     Ron Harper Jr.  TOR   2024  23  12.0       0.0        0.0
13415  13416     Justin Jackson  MIN   2024  28   3.0       0.0        0.0
13416  13417  Dmytro Skapintsev  NYK   2024  25   2.0       0.0        0.0
13417  13418      Javonte Smart  PH

In [71]:
#scraping data
def get_mvp_vote_data(url,mvp_data_storage):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
 
    this_year_votes={}
    current=""
    column=0
    for td in table.find_all('td'):
        
        if column%19==0:
            current=td.text.strip()
        elif column%19==4:
            points=float(td.text.strip())
            this_year_votes[current]=points
        column+=1
    mvp_data_storage.append(this_year_votes)

mvp_votes_url="https://www.basketball-reference.com/awards/awards_{}.html"
def gather_mvp_voting(start,end):
    mvp_data_storage=[]
    for i in range(start,end+1):
        get_mvp_vote_data(mvp_votes_url.format(i),mvp_data_storage)
    return mvp_data_storage
        
mvp_voting_data=gather_mvp_voting(start,2023)
print(mvp_voting_data)

[{'Karl Malone': 986.0, 'Michael Jordan': 957.0, 'Grant Hill': 376.0, 'Tim Hardaway': 238.0, 'Glen Rice': 134.0, 'Gary Payton': 105.0, 'Hakeem Olajuwon': 95.0, 'Patrick Ewing': 57.0, 'Anthony Mason': 7.0, "Shaquille O'Neal": 7.0, 'Scottie Pippen': 6.0, 'Alonzo Mourning': 5.0, 'Dikembe Mutombo': 4.0, 'Mitch Richmond': 4.0, 'John Stockton': 3.0, 'Charles Barkley': 2.0, 'Tom Gugliotta': 1.0, 'Allen Iverson': 1.0, 'Kevin Johnson': 1.0, 'Steve Smith': 1.0}, {'Michael Jordan': 1084.0, 'Karl Malone': 842.0, 'Gary Payton': 431.0, "Shaquille O'Neal": 311.0, 'Tim Duncan': 148.0, 'Tim Hardaway': 71.0, 'David Robinson': 36.0, 'Vin Baker': 24.0, 'Grant Hill': 23.0, 'Scottie Pippen': 14.0, 'Glen Rice': 7.0, 'Antoine Walker': 6.0, 'Jason Kidd': 5.0, 'John Stockton': 5.0, 'Mitch Richmond': 3.0, 'Reggie Miller': 2.0, 'Rik Smits': 2.0, 'Michael Finley': 1.0, 'Rod Strickland': 1.0}, {'Karl Malone': 827.0, 'Alonzo Mourning': 773.0, 'Tim Duncan': 740.0, 'Allen Iverson': 319.0, 'Jason Kidd': 159.0, "Shaquil

In [72]:
cur_year=0.0
mvp=""
cons=0.0
for i,v in enumerate(p_id.values):
    if p_id.at[i,"season"]=="2024":
        break
    
    if p_id.at[i,"Player"][-1]=="*":
        p_id.at[i,"Player"]= p_id.at[i,"Player"][:-1]
        
    
    if p_id.at[i,"season"]!=cur_year:
        cur_year=v[3]
        cur_year_data=mvp_voting_data[int(cur_year)-start]
        
        max_val=max(cur_year_data.values())
        
    if p_id.at[i,"Player"] in cur_year_data.keys():
    
        p_id.at[i,"mvp_votes"]=cur_year_data[p_id.at[i,"Player"]]
        
       
        if p_id.at[i,"mvp_votes"]==max_val:
            if p_id.at[i,"Player"]== mvp:
                print(p_id.at[i,"Player"])
                cons+=1
            else:
                cons=0
            mvp=p_id.at[i,"Player"]
            p_id.at[i,"cons_mvp"]=cons


p_id.to_csv("data/p_id.csv")

Tim Duncan
Steve Nash
LeBron James
LeBron James
Stephen Curry
Giannis Antetokounmpo
Nikola Jokić


In [73]:
for i in numerical_data:
        for k,n in enumerate(numerical_data[i]):
                if n=='':
                        numerical_data.at[k,i]=0.0
def find_number(components,cutoff):
    number=0
    for i in components:
        
        if i<cutoff:
            number+=1
    return number

In [74]:
# Import numpy
import numpy as np

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# Calculate the cumulative explained variance
pca = PCA()
pca.fit(numerical_data)
variance= pca.explained_variance_ratio_
cumulated_variance=np.cumsum(variance)
n_components=find_number(cumulated_variance, 0.99995)
print(n_components)


#exclude components
pca=PCA(n_components, random_state=12)
pca.fit(numerical_data)
pca_projection=pca.transform(numerical_data)

print(pca_projection)

30
[[ 7.18533459e+01 -1.41022782e+01  6.79889232e+00 ... -2.52907864e-02
  -3.95835531e-02 -1.16318692e-01]
 [ 7.10990250e+01 -1.35447637e+01  2.46673928e+00 ...  2.20926207e-01
   2.54638225e-01  5.96654564e-02]
 [ 6.66443254e+01 -1.35864148e+01  1.24937741e+00 ... -3.35524612e-01
  -7.41871171e-02 -5.14009162e-01]
 ...
 [-5.49087958e+01 -2.13632154e+01 -1.00106056e+01 ...  2.51087112e-01
  -4.88263431e-02 -4.94259107e-01]
 [-5.46825821e+01 -2.15240874e+01 -5.53836371e+00 ...  5.63594757e-02
   3.12426693e-01 -3.44615685e-01]
 [-5.53244874e+01 -2.21426719e+01 -5.12483044e+00 ...  2.25403791e-01
   3.62251444e-02 -4.25105309e-01]]


In [75]:
combined=pd.DataFrame(np.concatenate((p_id.values,numerical_data),axis=1),columns=np.append(p_id.columns,numerical_data.columns))

for i in combined:
    if i not in ["Rk","season","team","Player"] :
        for k,n in enumerate(combined[i]):
                if n=="":
                        combined.at[k,i]=0.0
             
for i in combined:
         if i not in ["Rk","season","team","Player"]:
                
                    combined[i]=np.array(combined[i], dtype=float)
                    combined[i]=np.array((combined[i]-min(combined[i]))/(max(combined[i])-min(combined[i])))
               

                


In [76]:
combined

,Rk,Player,team,season,Age,seed,cons_mvp,mvp_votes,G,GS,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1,Michael Jordan,CHI,1997,0.576923,0.000000,0.0,0.730534,0.964286,0.987952,...,0.108,0.262741,0.292818,0.168317,0.214286,0.493022,0.253820,0.319172,0.267724,0.152778
1,2,Karl Malone,UTA,1997,0.576923,0.000000,0.0,0.752672,0.964286,0.987952,...,0.135,0.312571,0.276243,0.217822,0.223214,0.490920,0.257920,0.315904,0.270117,0.194444
2,3,Glen Rice,CHH,1997,0.423077,0.357143,0.0,0.102290,0.928571,0.939759,...,0.112,0.227633,0.193370,0.108911,0.111607,0.490728,0.247111,0.320261,0.262339,0.166667
3,4,Shaquille O'Neal,LAL,1997,0.230769,0.214286,0.0,0.005344,0.595238,0.614458,...,0.211,0.251416,0.209945,0.158416,0.142857,0.488243,0.257920,0.330065,0.273706,0.215278
4,5,Mitch Richmond,SAC,1997,0.500000,0.571429,0.0,0.003053,0.952381,0.975904,...,0.214,0.176670,0.165746,0.178218,0.111607,0.485567,0.247484,0.336601,0.267724,0.173611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13413,13414,Danny Green,PHI,2024,0.692308,0.428571,0.0,0.000000,0.011905,0.000000,...,0.116,0.142695,0.232044,0.138614,0.151786,0.507933,0.254939,0.347495,0.276398,0.194444
13414,13415,Ron Harper Jr.,TOR,2024,0.192308,0.785714,0.0,0.000000,0.000000,0.000000,...,0.165,0.345413,0.403315,0.158416,0.299107,0.503345,0.275065,0.313725,0.283278,0.333333
13415,13416,Justin Jackson,MIN,2024,0.384615,0.142857,0.0,0.000000,0.011905,0.000000,...,0.159,0.202718,0.198895,0.138614,0.125000,0.493405,0.250839,0.322440,0.266527,0.173611
13416,13417,Dmytro Skapintsev,NYK,2024,0.269231,0.071429,0.0,0.000000,0.011905,0.000000,...,0.169,0.146093,0.204420,0.138614,0.129464,0.505257,0.245993,0.342048,0.267724,0.180556


In [100]:
combined.drop(["cons_mvp","mvp_votes","Rk","team","Player","season"],axis=1).values

array([[0.57692308, 0.        , 0.96428571, ..., 0.31917211, 0.2677236 ,
        0.15277778],
       [0.57692308, 0.        , 0.96428571, ..., 0.31590414, 0.27011666,
        0.19444444],
       [0.42307692, 0.35714286, 0.92857143, ..., 0.32026144, 0.26233922,
        0.16666667],
       ...,
       [0.38461538, 0.14285714, 0.01190476, ..., 0.32244009, 0.26652707,
        0.17361111],
       [0.26923077, 0.07142857, 0.01190476, ..., 0.34204793, 0.2677236 ,
        0.18055556],
       [0.23076923, 0.42857143, 0.        , ..., 0.33877996, 0.27789411,
        0.27083333]])

In [137]:
import torch
import torch.nn as nn
import torch.optim as optim
X=torch.tensor(combined.drop(["mvp_votes","Rk","team","Player","season"],axis=1).values[:12845],dtype=torch.float32)
y=torch.tensor(combined.mvp_votes.values,dtype=torch.float32)

In [138]:
model = nn.Sequential(
    nn.Linear(48, 15),
    nn.ReLU(),
    nn.Linear(15, 5),
    nn.ReLU(),
    nn.Linear(5, 1),
    nn.Sigmoid()
    
)

loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [142]:

n_epochs = 10
batch_size = 5
 
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        y_pred = model(X[i:i+batch_size]).reshape(batch_size)
        

        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 1.156802795776457e-07
Finished epoch 1, latest loss 1.6193753538118472e-07
Finished epoch 2, latest loss 9.708826098631107e-08
Finished epoch 3, latest loss 1.2920183678488684e-07
Finished epoch 4, latest loss 6.978731192930354e-08
Finished epoch 5, latest loss 7.254156741964835e-08
Finished epoch 6, latest loss 3.392568004301211e-08
Finished epoch 7, latest loss 3.137257209573363e-08
Finished epoch 8, latest loss 5.550208470594953e-08
Finished epoch 9, latest loss 5.978571238074437e-08


In [157]:
data2024=torch.tensor(combined.drop(["mvp_votes","Rk","team","Player","season"],axis=1).values[12845:],dtype=torch.float32)
mvp_pred=model(data2024).detach().numpy()

[3]


In [169]:
for i,v in enumerate(mvp_pred):
    combined.at[12845+i,"mvp_votes"]=v

In [170]:
print(combined[12845:])

          Rk                   Player team season       Age      seed  \
12845  12846         Alondes Williams  BRK   2023  0.192308  0.357143   
12846  12847              Joel Embiid  PHI   2024  0.423077  0.428571   
12847  12848              Luka Dončić  DAL   2024  0.230769  0.285714   
12848  12849    Giannis Antetokounmpo  MIL   2024  0.423077  0.142857   
12849  12850  Shai Gilgeous-Alexander  OKC   2024  0.269231  0.000000   
...      ...                      ...  ...    ...       ...       ...   
13413  13414              Danny Green  PHI   2024  0.692308  0.428571   
13414  13415           Ron Harper Jr.  TOR   2024  0.192308  0.785714   
13415  13416           Justin Jackson  MIN   2024  0.384615  0.142857   
13416  13417        Dmytro Skapintsev  NYK   2024  0.269231  0.071429   
13417  13418            Javonte Smart  PHI   2024  0.230769  0.428571   

       cons_mvp     mvp_votes         G        GS  ...   TOV%      USG%  \
12845       0.0  1.089996e-08  0.000000  0.00000